In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
pip install atproto

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.8/327.8 kB 5.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.9/671.9 kB 8.9 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install spacy
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 62.0 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires cloudpickle~=2.2.1, but you have cloudpickle 3.0.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 2.0.2 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 17.0.0 which is incompatible.
catboost 1.2.7 requires numpy<2.0,>=1.16.0, but you have numpy 2.0.2 which is incompatible.
gensim 4.3.

In [2]:
from atproto import Client
from kaggle_secrets import UserSecretsClient

# Retrieve Bluesky app password securely from Kaggle secrets
user_secrets = UserSecretsClient()
bluesky_password = user_secrets.get_secret("bluesky")

# Initialize the client
client = Client()

# Bluesky handle
handle = "katakurikpowder.bsky.social"

# Authenticate the client
try:
    client.login(handle, bluesky_password)
    print("Authenticated successfully!")
except Exception as e:
    print("Error during client initialization:", e)


Authenticated successfully!


In [8]:
try:
    # Fetch the custom feed using the identified URI
    data = client.app.bsky.feed.get_feed({
        'feed': 'at://did:plc:7mtqkeetxgxqfyhyi2dnyga2/app.bsky.feed.generator/aaadhh6hwvaca',
        'limit': 30,
    })

    # Extract and display the feed content
    feed = data.feed
    print(f"Fetched {len(feed)} posts from the custom feed:")
    for post in feed:
        post_view = post.post
        post_record = post_view.record
        if hasattr(post_record, 'text'):
            print(f"{post_view.author.handle}: {post_record.text}")
except Exception as e:
    print("Error fetching custom feed:", e)


Fetched 28 posts from the custom feed:
pleasesaveour.bsky.social: Is this a get out of jail free card that Biden just issued to corporations?

“Those that make good faith efforts to self-report, even if they do not qualify for a [full prosecution shield], could still receive substantial benefits"

www.justice.gov/opa/blog/tra...
liberalcurmudgeon.bsky.social: Not the FBI director yet, but Trump’s pick is on board with his campaign of political retribution.
sharkygnupluslinux.bsky.social: Kamala Harris got motherfucking DUSTED for a reason. Fuck the bourgeoisie. Vote for full blown communism or just don't fucking vote, you cretins have literally thrown america down the drain with your arrogance.
jimalb1.bsky.social: The Dumbocrats had 4 years to change many policies that they witness GOP use during Trump's 1st term.
They didn't do that! Most of these dummies mention reasons why Trump should be disqualified to run. They didn't rewrite any qualification policy to be President. They just h

In [3]:
from datetime import datetime, timedelta

def fetch_feed_weekly(client, feed_uri, start_date=None):
    """
    Fetch posts from a specific feed (weekly) using pagination.

    Parameters:
        client: Authenticated Bluesky client.
        feed_uri: The URI of the feed to fetch posts from.
        start_date: Optional datetime to stop fetching older posts.
    Returns:
        List of posts fetched from the feed.
    """
    all_posts = []
    cursor = None  # Start with no cursor (initial request)
    one_week_ago = datetime.now() - timedelta(days=7)

    if not start_date:
        start_date = one_week_ago

    while True:
        try:
            # Fetch posts from the feed with pagination
            data = client.app.bsky.feed.get_feed({
                'feed': feed_uri,
                'cursor': cursor,
                'limit': 30,  # Fetch 30 posts per page
            })

            # Process the feed
            feed = data.feed
            for post in feed:
                post_view = post.post
                post_record = post_view.record

                # Stop if the post is older than the desired date
                if hasattr(post_record, 'createdAt'):
                    post_date = datetime.fromisoformat(post_record.createdAt.replace("Z", "+00:00"))
                    if post_date < start_date:
                        print(f"Stopped fetching older posts. Fetched {len(all_posts)} posts total.")
                        return all_posts

                all_posts.append(post)

            # Handle pagination
            cursor = data.cursor
            if not cursor:
                break  # No more pages to fetch

        except Exception as e:
            print("Error during feed fetching:", e)
            break

    print(f"Finished fetching. Total posts fetched: {len(all_posts)}")
    return all_posts


# Run the weekly fetch
try:
    feed_uri = 'at://did:plc:7mtqkeetxgxqfyhyi2dnyga2/app.bsky.feed.generator/aaadhh6hwvaca'  # Replace with your feed URI
    weekly_posts = fetch_feed_weekly(client, feed_uri)
    print(f"Fetched {len(weekly_posts)} posts from the feed.")
except Exception as e:
    print("Error fetching weekly posts:", e)


Finished fetching. Total posts fetched: 959
Fetched 959 posts from the feed.


In [4]:
import spacy
from collections import Counter
import re

# Load spaCy language model
nlp = spacy.load("en_core_web_sm")

def extract_keywords_nlp(posts):
    """
    Extract meaningful keywords from posts using NLP techniques.
    """
    # Combine all post texts
    all_text = " ".join(
        post.post.record.text
        for post in posts
        if hasattr(post.post.record, 'text')
    )

    # Clean text: remove URLs, numbers, and short words
    cleaned_text = re.sub(r"http\S+|www\S+|\d+|\b\w{1,3}\b", "", all_text)

    # Process text using spaCy
    doc = nlp(cleaned_text)

    # Extract Named Entities (e.g., people, organizations, locations)
    named_entities = [ent.text for ent in doc.ents if ent.label_ in {"PERSON", "ORG", "GPE", "EVENT"}]

    # Extract Nouns, Verbs, and Adjectives
    pos_keywords = [token.text.lower() for token in doc if token.pos_ in {"NOUN", "VERB", "ADJ"} and not token.is_stop]

    # Combine entities and POS keywords
    all_keywords = named_entities + pos_keywords

    # Return the most common keywords
    return Counter(all_keywords).most_common(20)


# Analyze Keywords
try:
    nlp_keywords = extract_keywords_nlp(weekly_posts)  # Use the previously fetched posts
    print("Top Keywords from Weekly Feed:", nlp_keywords)
except Exception as e:
    print("Error during NLP keyword extraction:", e)


Top Keywords from Weekly Feed: [('Trump', 345), ('trump', 218), ('Kash Patel', 108), ('people', 94), ('MAGA', 73), ('Biden', 66), ('Patel', 49), ('think', 46), ('know', 45), ('Donald Trump', 43), ('time', 43), ('years', 42), ('media', 38), ('going', 37), ('said', 37), ('right', 35), ('director', 32), ('want', 30), ('voters', 30), ('good', 29)]


In [5]:
import random

def generate_post_from_keywords(keywords, template=None):
    """
    Generate a custom post using top keywords and optional templates.
    """
    # Use a default template if none is provided
    if not template:
        template = random.choice([
            "Breaking news: {entity1} says {entity2} is {action}—what are your thoughts?",
            "Is {entity1} the key to solving {entity2}? Let's discuss!",
            "People are talking about {entity1} and {entity2}. Where do you stand?",
            "Latest update: {entity1} claims {entity2} will {action}. Agree or disagree?",
            "It seems like {entity1} vs. {entity2} is heating up. Whose side are you on?",
        ])

    # Select random entities, actions, and topics from the keywords
    entities = [kw[0] for kw in keywords if kw[0].istitle()]  # Proper nouns
    actions = [kw[0] for kw in keywords if kw[0].islower()]   # Verbs or lower-case nouns
    random.shuffle(entities)
    random.shuffle(actions)

    # Ensure enough entities and actions
    entity1 = entities[0] if len(entities) > 0 else "Someone"
    entity2 = entities[1] if len(entities) > 1 else "something"
    action = actions[0] if len(actions) > 0 else "doing something interesting"

    # Format the template with selected keywords
    return template.format(entity1=entity1, entity2=entity2, action=action)


# Generate a post
try:
    generated_post = generate_post_from_keywords(nlp_keywords)
    print("Generated Post:", generated_post)
except Exception as e:
    print("Error generating post:", e)


Generated Post: It seems like Donald Trump vs. Kash Patel is heating up. Whose side are you on?


In [6]:
import random
from collections import defaultdict
import spacy

# Load spaCy NLP model
nlp = spacy.load("en_core_web_sm")

def identify_entity_relationships(posts):
    """
    Analyze relationships between entities by co-occurrences in posts.
    """
    relationship_counts = defaultdict(int)
    for post in posts:
        if hasattr(post.post.record, 'text'):
            text = post.post.record.text
            doc = nlp(text)
            entities = [ent.text for ent in doc.ents if ent.label_ in {"PERSON", "ORG"}]
            for i in range(len(entities)):
                for j in range(i + 1, len(entities)):
                    pair = tuple(sorted([entities[i], entities[j]]))
                    relationship_counts[pair] += 1
    return sorted(relationship_counts.items(), key=lambda x: -x[1])


def generate_contextual_post(relationships, keywords):
    """
    Generate a post that considers relationships and context.
    """
    # Use top relationship pairs
    if relationships:
        top_relationship = relationships[0][0]  # Most common pair
        entity1, entity2 = top_relationship
    else:
        entity1, entity2 = "Someone", "Something"

    # Choose a template based on common contexts
    templates = [
        "{entity1} has nominated {entity2}—what do you think about this move?",
        "The partnership between {entity1} and {entity2} is stirring debate. What's your take?",
        "Breaking news: {entity1} and {entity2} are making headlines together!",
        "Is {entity1} supporting {entity2} the right move? Share your thoughts.",
        "What does the alliance between {entity1} and {entity2} mean for the future?",
    ]

    # Randomly pick a template
    template = random.choice(templates)
    return template.format(entity1=entity1, entity2=entity2)


# Analyze relationships
try:
    relationships = identify_entity_relationships(weekly_posts)
    print("Top Relationships:", relationships[:5])  # Show top 5 relationships

    # Generate a contextual post
    contextual_post = generate_contextual_post(relationships, nlp_keywords)
    print("Generated Contextual Post:", contextual_post)
except Exception as e:
    print("Error generating contextual post:", e)


Top Relationships: [(('FBI', 'Trump'), 71), (('Trump', 'Trump'), 64), (('FBI', 'Kash Patel'), 62), (('Kash Patel', 'Trump'), 40), (('Patel', 'Trump'), 37)]
Generated Contextual Post: Breaking news: FBI and Trump are making headlines together!


In [12]:
from datetime import datetime

def post_to_account(client, post_text):
    """
    Post generated content to your Bluesky account.
    """
    try:
        # Construct the post record
        post_record = {
            "$type": "app.bsky.feed.post",
            "text": post_text,
            "createdAt": datetime.utcnow().isoformat() + "Z"  # Add UTC timestamp
        }

        # Post the content to Bluesky
        response = client.app.bsky.feed.post.create(
            repo=client.me.did,  # Use the DID of the authenticated user
            record=post_record   # The post record with required parameters
        )

        print("Post successful! Here's the post data:")
        print(response)
    except Exception as e:
        print("Error posting to account:", e)


# Example usage
try:
    generated_post = "Breaking news: FBI and Trump are making headlines together!"  # Replace with your generated post
    post_to_account(client, generated_post)
except Exception as e:
    print("Error during posting workflow:", e)


Post successful! Here's the post data:
uri='at://did:plc:wp2rgshljsbqpemy2bockgyh/app.bsky.feed.post/3lcb2sy4na722' cid='bafyreihwa2oiw5nikgdrbonu6ifle3gicj2dn6tpwlegvyx4eturwxghje'
